Step 1: Create df of urls w/how many pages are in the topic.

Step 2: Create system for filling out those two things in new df.

## The Plan
- Gather each piece of data that we care about
- Move through each page that exists.
- End at the end of the range of pages
- Have all rows of posts added to the posts_df
- Change the row value of "Topic_scrapped" to "True" in url_df

## Imports and Function definitions

In [1]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np
%matplotlib inline
import requests
from time import sleep
from bs4 import BeautifulSoup
import json
from time import time
from datetime import datetime
from warnings import filterwarnings
filterwarnings('ignore')

In [16]:
url_df.shape[0]

499867

In [2]:
def get_html(url):
    return BeautifulSoup(requests.get(url).content)

In [3]:
def urls_with_numbers(forum_page_url, forum_page_num = False):
    forum_soup     = get_html(forum_page_url)     # Input URL get Soup
    
    # List of urls of topics in forum page
    topic_url_list = ['https://us.battle.net' + topic.attrs['href'] 
        for topic in forum_soup.find_all(attrs={'class': "ForumTopic"})]
    topics         = []
    count          = 0
    
    # for i in all the forum topic infos
    for forum_topic in forum_soup.find_all(attrs={'class': "ForumTopic"}):
        # Turning each individual info bit into a dict
        posts_num   = json.loads(forum_topic.attrs['data-forum-topic']
            )['lastPosition'] 
        
        # Num of posts in Topic divided by num allowed per page +1 for 1st
        topic_pages = posts_num//20 + 1 
        
        if posts_num%20 != 0 and posts_num > 20:# If there's a remainder
            topic_pages += 1                    # Add remainder page
        topics.append({
            'url'      : topic_url_list[count],
            'pages'    : topic_pages,
            'Forum_num': forum_page_num
            })
        count += 1
    return topics # Returns list of dicts


In [4]:
def validate(post_item, datum_type):
    the_type = type(post_item)
    value = 'N/a'
    if the_type == type(None):
        value = "None"
    else:
        if   datum_type == "prof_link":
            value = post_item.attrs['href']
        elif datum_type == "auth_posts":
            value = post_item.text
        elif datum_type == "auth_img":
            value = post_item.find('img').attrs['src']
        #elif datum_type == "":
        #    value = post_item.
        else:
            value = "Terra, a mistake occured & we love you. <3"
    return value

        #elif datum_type == "":
        #    value = post_item.
        
        
        
                #    topic_statuses = soup.find(attrs = {"class":
                #'TopicPost'}).attrs['data-topic']

In [5]:
def post_dictionary(post, title, forum_page, topic_url, topic_statuses, page):
    return {'text'      : post.find('div', {"class" :'TopicPost-bodyContent'}).text,
            'date'      : post.find('a',{"class" :'TopicPost-timestamp'}).attrs['data-tooltip-content'],
            'ids_dict'  : post.attrs['data-topic-post'],
            'post_num'  : post.attrs['id'],#Post num in topic
            'title'     : title,# Title of Topic
            'forum_page': forum_page,# Page in the forum
            'topic_url' : topic_url,
            'statuses'  : topic_statuses,
            'prof_link' : validate(post.find(attrs ={"class":'Author-avatar '}), 'prof_link'),
            'auth_posts': validate(post.find('a',{"class" :'Author-posts'}),'auth_posts'),
            'auth_img'  : validate(post.find(attrs ={"class":'Author-avatar '}), 'auth_img'),
            'topic_page': page}

## Getting the initial information
My first step is getting the urls and any other needed information from the 

In [ ]:
dict_list = []
b_url = 'https://us.battle.net/forums/en/overwatch/22813879/?page='
for attempt in range(5):
    try:
        for forum_num in range(9999):
            current_list = urls_with_numbers(b_url + str(forum_num),
                forum_num)
            
            dict_list += current_list
            if forum_num % 100 == 0:
                print(forum_num, 'Added', 
                    datetime.fromtimestamp(time()
                    ).strftime('%H:%M:%S'))
    except:
        print("---------")
        print('Error', attempt)
        print("---------")
        sleep(100)
    pd.DataFrame(dict_list).to_csv('./data/urls' + str(attempt) + '.csv')

## From urls to full scrapes

In [6]:
url_df = pd.DataFrame.from_csv('./../data/urls_v2.csv')

url_df['Topic_scrapped'] = False

url_df.reset_index(inplace = True)

url_df.head()#.append()

,original_index,Forum_num,pages,url,Topic_scrapped
0,48,1,1,https://us.battle.net/forums/en/overwatch/topi...,False
1,49,1,1,https://us.battle.net/forums/en/overwatch/topi...,False
2,50,1,238,https://us.battle.net/forums/en/overwatch/topi...,False
3,51,1,6,https://us.battle.net/forums/en/overwatch/topi...,False
4,52,1,10,https://us.battle.net/forums/en/overwatch/topi...,False


In [7]:
posts_df = pd.DataFrame(columns = ['text',
                            'date',
                            'ids_dict',
                            'post_num',
                            'auth_posts',
                            'prof_link',
                            'title',
                            'forum_page',
                            'statuses',
                            'topic_url',
                            'topic_page'])

#posts_df = posts_df.append({
#    'date' : 'test_teeessest'},
#    ignore_index= True)

posts_df

,text,date,ids_dict,post_num,auth_posts,prof_link,title,forum_page,statuses,topic_url,topic_page


## Extracting the data the want from each post of the html

In [16]:
def scrape_from_urldf(topic_num_start, topic_num_stop, url_df = url_df, posts_df = posts_df):
    posts_df = posts_df
    for topic_num in range(topic_num_start,topic_num_stop):
        if url_df.loc[topic_num,'Topic_scrapped'] == False:
            pages     = url_df.loc[topic_num,'pages'] + 1
            topic_url = url_df.loc[topic_num,'url']
            forum_page= url_df.loc[topic_num,'Forum_num']
            for page in range(1,pages):

                # Keep us updated & Defining things @ page level
                print('Forum:', url_df.loc[topic_num,'Forum_num'],
                      'Page:', page,'/', pages - 1, 
                      "Time:",datetime.fromtimestamp(time()
                            ).strftime('%H:%M:%S'),
                      'URL:',url_df.loc[topic_num,'url'][47:])
                soup = get_html(topic_url + '?page=' + str(page))

                # Define the title one time only.
                if page == 1:
                    title = soup.find(attrs={'class':'Topic-title'}).text

                # Will obtain the topic status one time, if it exists.
                try:
                    topic_statuses = soup.find(attrs = {"class":
                    'TopicPost'}).attrs['data-topic']
                except:
                    topic_statuses = 'No Statuses'
                Topic_Posts = soup.find_all(attrs={"class":'TopicPost'})
                for post in Topic_Posts:
                    posts_df = posts_df.append(post_dictionary(post, 
                        title, forum_page, topic_url, topic_statuses, page), 
                        ignore_index = True)
                sleep(1)
            url_df.loc[topic_num,'Topic_scrapped'] = True
        else:
            pass
    return

In [19]:
scrape_from_urldf(0, 3, url_df = url_df)

Forum: 1 Page: 1 / 238 Time: 06:56:09 URL: /20761088072
Forum: 1 Page: 2 / 238 Time: 06:56:16 URL: /20761088072
Forum: 1 Page: 3 / 238 Time: 06:56:22 URL: /20761088072


KeyboardInterrupt: 

In [20]:
posts_df

,text,date,ids_dict,post_num,auth_posts,prof_link,title,forum_page,statuses,topic_url,topic_page


In [11]:
for attempt in range(10):
    try:
        scrape_from_urldf(0, 2, url_df = url_df)
    except:
        print("---------")
        print('Error', attempt)
        print("---------")
        sleep(100)
    posts_df.to_csv('../data/posts_' + str(topic_num) + '.csv')
    print('SAVED AT',topic_num)


Forum: 1 Page: 1 / 1 Time: 06:54:31 URL: /20744265634
---------
Error 0
---------


KeyboardInterrupt: 

In [ ]:
for attempt in range(10):
    try:
        for topic_num in range(6330,url_df.shape[0] + 1):
            if url_df.loc[topic_num,'Topic_scrapped'] == False:
                pages     = url_df.loc[topic_num,'pages'] + 1
                topic_url = url_df.loc[topic_num,'url']
                forum_page= url_df.loc[topic_num,'Forum_num']
                for page in range(1,pages):
                    
                    # Keep us updated & Defining things @ page level
                    print('Forum:', url_df.loc[topic_num,'Forum_num'],
                          'Page:', page,'/', pages - 1, 
                          "Time:",datetime.fromtimestamp(time()
                                ).strftime('%H:%M:%S'),
                          'URL:',url_df.loc[topic_num,'url'][47:], 
                          'Topic #:', topic_num)
                    soup = get_html(topic_url + '?page=' + str(page))
                    
                    # Define the title one time only.
                    if page == 1:
                        title = soup.find(attrs={'class':'Topic-title'}).text
                        
                    # Will obtain the topic status one time, if it exists.
                    try:
                        topic_statuses = soup.find(attrs = {"class":
                        'TopicPost'}).attrs['data-topic']
                    except:
                        topic_statuses = 'No Statuses'
                    Topic_Posts = soup.find_all(attrs={"class":'TopicPost'})
                    for post in Topic_Posts:
                        posts_df = posts_df.append(post_dictionary(post, 
                            title, forum_page, topic_url, topic_statuses, page), 
                            ignore_index = True)
                    sleep(3)
                url_df.loc[topic_num,'Topic_scrapped'] = True
            else:
                pass
    except:
        print("---------")
        print('Error', attempt)
        print("---------")
        sleep(100)
        posts_df.to_csv('../data/posts_' + str(topic_num) + '.csv')
    posts_df.to_csv('../data/posts_' + str(topic_num) + '.csv')
    print('SAVED AT',topic_num)


Forum: 127 Page: 1 / 7 Time: 19:06:23 URL: /20744874974 Topic #: 6331
Forum: 127 Page: 2 / 7 Time: 19:06:29 URL: /20744874974 Topic #: 6331
Forum: 127 Page: 3 / 7 Time: 19:06:35 URL: /20744874974 Topic #: 6331
Forum: 127 Page: 4 / 7 Time: 19:06:41 URL: /20744874974 Topic #: 6331
Forum: 127 Page: 5 / 7 Time: 19:06:47 URL: /20744874974 Topic #: 6331
Forum: 127 Page: 6 / 7 Time: 19:06:53 URL: /20744874974 Topic #: 6331
Forum: 127 Page: 7 / 7 Time: 19:06:57 URL: /20744874974 Topic #: 6331
Forum: 127 Page: 1 / 1 Time: 19:07:01 URL: /20761657470 Topic #: 6332
Forum: 127 Page: 1 / 1 Time: 19:07:05 URL: /20761767377 Topic #: 6333
Forum: 127 Page: 1 / 1 Time: 19:07:09 URL: /20761797519 Topic #: 6334
Forum: 127 Page: 1 / 1 Time: 19:07:14 URL: /20761787512 Topic #: 6335
Forum: 127 Page: 1 / 1 Time: 19:07:18 URL: /20761597352 Topic #: 6336
Forum: 127 Page: 1 / 1 Time: 19:07:22 URL: /20761787511 Topic #: 6337
Forum: 127 Page: 1 / 1 Time: 19:07:27 URL: /20761777451 Topic #: 6338
Forum: 127 Page: 1 /

Forum: 129 Page: 1 / 1 Time: 19:15:51 URL: /20761727407 Topic #: 6408
Forum: 129 Page: 1 / 1 Time: 19:15:56 URL: /20761657448 Topic #: 6409
Forum: 129 Page: 1 / 1 Time: 19:16:01 URL: /20761627392 Topic #: 6410
Forum: 129 Page: 1 / 1 Time: 19:16:05 URL: /20761747494 Topic #: 6411
Forum: 129 Page: 1 / 1 Time: 19:16:09 URL: /20761657446 Topic #: 6412
Forum: 129 Page: 1 / 1 Time: 19:16:13 URL: /20761677430 Topic #: 6413
Forum: 129 Page: 1 / 1 Time: 19:16:18 URL: /20759181200 Topic #: 6414
Forum: 129 Page: 1 / 1 Time: 19:16:22 URL: /20761617380 Topic #: 6415
Forum: 129 Page: 1 / 1 Time: 19:16:26 URL: /20759210508 Topic #: 6416
Forum: 129 Page: 1 / 1 Time: 19:16:32 URL: /20761807478 Topic #: 6417
Forum: 129 Page: 1 / 1 Time: 19:16:37 URL: /20761697415 Topic #: 6418
Forum: 129 Page: 1 / 1 Time: 19:16:41 URL: /20761797511 Topic #: 6419
Forum: 129 Page: 1 / 1 Time: 19:16:45 URL: /20761647308 Topic #: 6420
Forum: 129 Page: 1 / 1 Time: 19:16:50 URL: /20761627390 Topic #: 6421
Forum: 129 Page: 1 /

Forum: 130 Page: 2 / 2 Time: 19:25:18 URL: /20761757397 Topic #: 6496
Forum: 131 Page: 1 / 4 Time: 19:25:22 URL: /20761747443 Topic #: 6497
Forum: 131 Page: 2 / 4 Time: 19:25:28 URL: /20761747443 Topic #: 6497
Forum: 131 Page: 3 / 4 Time: 19:25:34 URL: /20761747443 Topic #: 6497
Forum: 131 Page: 4 / 4 Time: 19:25:39 URL: /20761747443 Topic #: 6497
Forum: 131 Page: 1 / 1 Time: 19:25:42 URL: /20761647491 Topic #: 6498
Forum: 131 Page: 1 / 1 Time: 19:25:47 URL: /20761657379 Topic #: 6499
Forum: 131 Page: 1 / 4 Time: 19:25:51 URL: /20760988155 Topic #: 6500
Forum: 131 Page: 2 / 4 Time: 19:25:57 URL: /20760988155 Topic #: 6500
Forum: 131 Page: 3 / 4 Time: 19:26:03 URL: /20760988155 Topic #: 6500
Forum: 131 Page: 4 / 4 Time: 19:26:08 URL: /20760988155 Topic #: 6500
Forum: 131 Page: 1 / 1 Time: 19:26:12 URL: /20761727394 Topic #: 6501
Forum: 131 Page: 1 / 1 Time: 19:26:17 URL: /20761707463 Topic #: 6502
Forum: 131 Page: 1 / 1 Time: 19:26:21 URL: /20761627385 Topic #: 6503
Forum: 131 Page: 1 /

Forum: 132 Page: 1 / 1 Time: 19:48:02 URL: /20761667400 Topic #: 6567
Forum: 132 Page: 1 / 3 Time: 19:48:22 URL: /20761637269 Topic #: 6568
Forum: 132 Page: 2 / 3 Time: 19:48:56 URL: /20761637269 Topic #: 6568
Forum: 132 Page: 3 / 3 Time: 19:49:17 URL: /20761637269 Topic #: 6568
Forum: 132 Page: 1 / 1 Time: 19:49:27 URL: /20761657324 Topic #: 6569
Forum: 132 Page: 1 / 1 Time: 19:49:51 URL: /20761637328 Topic #: 6570
Forum: 132 Page: 1 / 3 Time: 19:50:10 URL: /20761647468 Topic #: 6571
Forum: 132 Page: 2 / 3 Time: 19:50:16 URL: /20761647468 Topic #: 6571
Forum: 132 Page: 3 / 3 Time: 19:50:22 URL: /20761647468 Topic #: 6571
Forum: 132 Page: 1 / 1 Time: 19:50:27 URL: /20761816974 Topic #: 6572
Forum: 132 Page: 1 / 1 Time: 19:50:55 URL: /20761647479 Topic #: 6573
Forum: 132 Page: 1 / 1 Time: 19:51:25 URL: /20761687595 Topic #: 6574
Forum: 132 Page: 1 / 1 Time: 19:51:56 URL: /20761707457 Topic #: 6575
Forum: 132 Page: 1 / 1 Time: 19:52:12 URL: /20761767355 Topic #: 6576
Forum: 132 Page: 1 /

Forum: 133 Page: 3 / 3 Time: 20:26:09 URL: /20761647180 Topic #: 6630
Forum: 133 Page: 1 / 1 Time: 20:26:19 URL: /20761827431 Topic #: 6631
Forum: 133 Page: 1 / 1 Time: 20:26:38 URL: /20761797491 Topic #: 6632
Forum: 133 Page: 1 / 1 Time: 20:26:50 URL: /20761657426 Topic #: 6633
Forum: 133 Page: 1 / 1 Time: 20:27:03 URL: /20761817501 Topic #: 6634
Forum: 133 Page: 1 / 1 Time: 20:27:18 URL: /20761597321 Topic #: 6635
Forum: 133 Page: 1 / 1 Time: 20:27:33 URL: /20761777402 Topic #: 6636
Forum: 133 Page: 1 / 1 Time: 20:28:09 URL: /20761747459 Topic #: 6637
Forum: 133 Page: 1 / 3 Time: 20:28:23 URL: /20760957913 Topic #: 6638
Forum: 133 Page: 2 / 3 Time: 20:28:55 URL: /20760957913 Topic #: 6638
Forum: 133 Page: 3 / 3 Time: 20:29:12 URL: /20760957913 Topic #: 6638
Forum: 133 Page: 1 / 1 Time: 20:29:22 URL: /20761617355 Topic #: 6639
Forum: 133 Page: 1 / 1 Time: 20:29:35 URL: /20761647435 Topic #: 6640
Forum: 133 Page: 1 / 1 Time: 20:30:00 URL: /20761677379 Topic #: 6641
Forum: 133 Page: 1 /

In [9]:
posts_df.to_csv('../data/posts_' + str(topic_num) + '.csv')

In [14]:
url_df.loc[6000:6005]
#topic_num
url_df.to_csv('../data/urls_pull_3', index= False)

In [8]:
posts_df = pd.DataFrame.from_csv('../data/posts_1000.csv')

In [10]:
posts_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16946 entries, 0 to 16945
Data columns (total 12 columns):
text          16946 non-null object
date          16946 non-null object
ids_dict      16946 non-null object
post_num      16946 non-null object
auth_posts    16946 non-null object
prof_link     16946 non-null object
title         16946 non-null object
forum_page    16946 non-null int64
statuses      16946 non-null object
topic_url     16946 non-null object
topic_page    16946 non-null int64
auth_img      16946 non-null object
dtypes: int64(2), object(10)
memory usage: 1.7+ MB


In [13]:
posts_df.shape[0] > 750000

False

In [118]:
url_df.loc[number,'url'][47:]

'/20761088072'

In [109]:
soup.find(attrs = {"class":
                'TopicPost'}).attrs

{'class': ['TopicPost', 'is-highly-rated', ''],
 'id': 'post-21',
 'data-topic-post': '{"id":"207592902941","valueVoted":0,"rank":{"voteUp":293,"voteDown":16},"author":{"id":"207576398916","name":"TheArchangel"}}'}

In [105]:
posts_df
#posts_df = posts_df.append({
#    'date' : 'test_teeessest'},
#    ignore_index= True)

,text,date,ids_dict,post_num,auth_posts,prof_link,title,forum_page,statuses,topic_url,auth_img
0,Welcome to the General Discussion forum! We en...,05/23/2016 09:32 PM,"{""id"":""207423780179"",""valueVoted"":0,""rank"":{""v...",post-1,None,None,Welcome to General Discussion - Please Read!,9999,"{ ""sticky"":""true"",""featured"":""false"",""locked"":...",https://us.battle.net/forums/en/overwatch/topi...,None
1,"In addition to the forum Code of Conduct, here...",05/23/2016 09:33 PM,"{""id"":""207423980450"",""valueVoted"":0,""rank"":{""v...",post-2,None,None,Welcome to General Discussion - Please Read!,9999,"{ ""sticky"":""true"",""featured"":""false"",""locked"":...",https://us.battle.net/forums/en/overwatch/topi...,None
2,To help us get the most of out of your feedbac...,05/23/2016 09:36 PM,"{""id"":""207423481262"",""valueVoted"":0,""rank"":{""v...",post-1,None,None,Bug Report and Technical Support Forums,9999,"{ ""sticky"":""true"",""featured"":""false"",""locked"":...",https://us.battle.net/forums/en/overwatch/topi...,None


In [72]:
# Get HTML and save it as Soup
title = test_soup.find(attrs={'class':'Topic-title'}).text
topic_statuses = test_soup.find(attrs = {"class" :'TopicPost'}
    ).attrs['data-topic']
Topic_Posts = test_soup.find_all(attrs = {"class" :'TopicPost'})
test_list_dicts = []
count = 0
for post in Topic_Posts:
    test_dict = {
    'text'      : post.find('div',
        {"class" :'TopicPost-bodyContent'}).text,
    'date'      : post.find('a',
        {"class" :'TopicPost-timestamp'}
        ).attrs['data-tooltip-content'],# Date of (unedited) post
    'ids_dict'  : post.attrs['data-topic-post'],# Author info & votes
    'post_num'  : post.attrs['id'],# Post number in the topic
    'title'     : title,# Title of Topic
    'forum_page': test_F_page,# Page in the forum
    'topic_url' : test_url,
    'statuses'  : topic_statuses,
    'prof_link' : validate(post.find(attrs = {"class" :
        'Author-avatar '}), 'prof_link'),
    'auth_posts': validate(post.find('a',
        {"class" :'Author-posts'}),'auth_posts'),
    'auth_img'  : validate(post.find(attrs = {"class" :
        'Author-avatar '}), 'auth_img')
    }
    test_list_dicts.append(test_dict)

pd.DataFrame(test_list_dicts, columns = ['text',
    'date', 'ids_dict', 'post_num', 'auth_posts',
    'auth_img', 'prof_link', 'title', 'forum_page',
    'statuses', 'topic_url'])

0
1
2
3
4
5
6
7
8
9
10
11
12


,text,date,ids_dict,post_num,auth_posts,auth_img,prof_link,title,forum_page,statuses,topic_url
0,"i can join a game, play for a few minutes and ...",05/23/2016 11:48 PM,"{""id"":""207422680682"",""valueVoted"":0,""rank"":{""v...",post-1,\n3 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/A...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
1,Same issue here. I'm on the west coast of the ...,05/24/2016 12:09 AM,"{""id"":""207424974255"",""valueVoted"":0,""rank"":{""v...",post-2,\n58 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/I...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
2,"Same here, has happened 3 times now",05/24/2016 12:10 AM,"{""id"":""207424374265"",""valueVoted"":0,""rank"":{""v...",post-3,\n3 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/R...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
3,Happening all over -- East Coast US here. Jus...,05/24/2016 12:13 AM,"{""id"":""207425074264"",""valueVoted"":0,""rank"":{""v...",post-4,\n1763 posts\n,None,None,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
4,Same here...,05/24/2016 12:19 AM,"{""id"":""207425474264"",""valueVoted"":0,""rank"":{""v...",post-5,\n21 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/N...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
5,"East Coast US, started the game > options > di...",05/24/2016 12:24 AM,"{""id"":""207425074284"",""valueVoted"":0,""rank"":{""v...",post-6,\n127 posts\n,None,None,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
6,"seems to just be server problems, which is exp...",05/24/2016 12:28 AM,"{""id"":""207424374309"",""valueVoted"":0,""rank"":{""v...",post-7,\n37 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/s...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
7,I finally stopped D/Cing but all I do is watch...,05/24/2016 12:31 AM,"{""id"":""207424574311"",""valueVoted"":0,""rank"":{""v...",post-8,\n31 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/T...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
8,Its launch day probably another DDOT attack,05/24/2016 12:37 AM,"{""id"":""207424974299"",""valueVoted"":0,""rank"":{""v...",post-9,\n290 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/L...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...
9,"i cant even join a game, i ""join"" then gets st...",05/24/2016 12:47 AM,"{""id"":""207424574349"",""valueVoted"":0,""rank"":{""v...",post-10,\n59 posts\n,https://blzgdapipro-a.akamaihd.net/game/unlock...,https://playoverwatch.com/en-us/career/pc/us/L...,D/C,9999,"{ ""sticky"":""false"",""featured"":""false"",""locked""...",https://us.battle.net/forums/en/overwatch/topi...


In [75]:
url_df.head()

,original_index,Forum_num,pages,url,Topic_scrapped
0,48,1,1,https://us.battle.net/forums/en/overwatch/topi...,False
1,49,1,1,https://us.battle.net/forums/en/overwatch/topi...,False
2,50,1,238,https://us.battle.net/forums/en/overwatch/topi...,False
3,51,1,6,https://us.battle.net/forums/en/overwatch/topi...,False
4,52,1,10,https://us.battle.net/forums/en/overwatch/topi...,False


In [78]:
for row_maybe, something in url_df.index:
    print(row_maybe, something)

TypeError: cannot unpack non-iterable int object